In [1]:
import pandas as pd


In [7]:
df = pd.read_excel(r'/Users/dhruvnovaims/Library/Mobile Documents/com~apple~CloudDocs/Documents/GitHub/climate-econometrics/countries/portugal/purchases_paper/datasets/final_data_mun.xlsx')


In [9]:
df

,Unnamed: 0,time,concelho,hpi,hpi_log,purchases,purchases_log,ur,fwi,high_fire_days,...,Medit_factor_t2m_L9,Medit_factor_t2m_L10,Medit_factor_t2m_L11,Medit_factor_t2m_L12,CCA_2,hpi_log_L1,ur_L1,purchases_log_L1,Linear_Trend,Quadratic_Trend
0,2908,2011Q1,castro daire,102.535801,4.630212,8.308960e+05,13.628497,0.060671,1.578201,0.000000,...,0.380323,-0.198971,-0.700392,-1.103489,1803,4.687124,0.060398,13.851949,1,1
1,2688,2011Q1,cascais,102.875767,4.633522,7.481642e+07,18.129664,0.065630,2.624597,0.000000,...,0.380323,-0.198971,-0.700392,-1.103489,1105,4.650176,0.066155,18.331527,1,1
2,2864,2011Q1,castelo de vide,101.528966,4.620344,3.720510e+05,12.824620,0.055017,3.182670,0.000000,...,0.380323,-0.198971,-0.700392,-1.103489,1205,4.639066,0.056345,12.998404,1,1
3,2204,2011Q1,braga,101.928180,4.624268,4.728596e+07,17.670746,0.083660,2.544331,0.000000,...,0.380323,-0.198971,-0.700392,-1.103489,303,4.629186,0.081808,17.900426,1,1
4,1368,2011Q1,armamar,102.535801,4.630212,4.064090e+05,12.905171,0.085280,1.512219,0.000000,...,0.380323,-0.198971,-0.700392,-1.103489,1801,4.687124,0.081909,13.018265,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11115,7655,2021Q4,palmela,172.581089,5.150867,2.579714e+07,17.063384,0.043367,12.380932,0.591664,...,-1.066671,-1.066995,-0.921213,-0.442357,1508,5.112920,0.043269,17.029011,40,1600
11116,7611,2021Q4,pacos de ferreira,130.528326,4.871590,1.695387e+07,16.638206,0.045452,4.181444,0.000000,...,-1.066671,-1.066995,-0.921213,-0.442357,1309,4.851249,0.049805,16.558571,40,1600
11117,7567,2021Q4,ovar,162.203479,5.088852,1.938856e+07,16.775122,0.041553,4.274507,0.000000,...,-1.066671,-1.066995,-0.921213,-0.442357,115,5.036607,0.053729,16.783389,40,1600
11118,8183,2021Q4,peso da regua,121.692245,4.801495,5.154025e+06,15.452136,0.110009,4.637445,0.000000,...,-1.066671,-1.066995,-0.921213,-0.442357,1708,4.785816,0.109058,15.557820,40,1600


In [8]:
# Step 1: Create linear trends that are specific to each municipality
df['Linear_Trend'] = df.groupby('CCA_2').cumcount() + 1  # Creates a linear trend for each municipality

# Step 2: Create quadratic trends that are specific to each municipality
df['Quadratic_Trend'] = df['Linear_Trend'] ** 2  # Quadratic trend is the square of the linear trend

In [5]:
%load_ext autoreload
%autoreload 2
import panelvar

In [ ]:
## ---------------- 1. core variable sets (edit if names change] -------------
endog        =["purchases_log_L1", "ur_L1", "hpi_log_L1"]
exo_base     = [                       # contemporaneous exogenous terms
  "fwi", "high_fire_days", "historical_dev_tp",
  "tp_std_mm_hour", "historical_dev_t2m", "t2m_std"
]

p = 2
exo_with_lags = exo_base.copy()
for lag in range(p):

    col_lags = [f"{exo}_L{lag+1}" for exo in exo_base]
    exo_with_lags = exo_with_lags + col_lags


['fwi',
 'high_fire_days',
 'historical_dev_tp',
 'tp_std_mm_hour',
 'historical_dev_t2m',
 't2m_std',
 'fwi_L1',
 'high_fire_days_L1',
 'historical_dev_tp_L1',
 'tp_std_mm_hour_L1',
 'historical_dev_t2m_L1',
 't2m_std_L1',
 'fwi_L2',
 'high_fire_days_L2',
 'historical_dev_tp_L2',
 'tp_std_mm_hour_L2',
 'historical_dev_t2m_L2',
 't2m_std_L2']

In [6]:
# mun_fe_series_basic <- pvarfeols(
#     dependent_vars = c('purchases_log_L1','ur_L1', 'hpi_log_L1'),
#     lags = 4,
#     exog_vars = c('Linear_Trend', 'Quadratic_Trend' , 'fwi', 'high_fire_days', 'historical_dev_tp',
#        'tp_std_mm_hour', 
#        'historical_dev_t2m', 't2m_std'),
#   data = df_ts_series_as_dataframe,
#   panel_identifier = c('CCA_2', 'time'),
model_fe = panelvar.pvarfeols(
    dependent_vars=['purchases_log_L1','ur_L1', 'hpi_log_L1'],
    lags=2,
    exog_vars= ['Linear_Trend', 'Quadratic_Trend' , 'fwi', 'high_fire_days', 'historical_dev_tp',
       'tp_std_mm_hour', 
       'historical_dev_t2m', 't2m_std'],data=df, panel_identifier=('CCA_2', 'time')

)

/Users/dhruvnovaims/Library/Mobile Documents/com~apple~CloudDocs/Documents/GitHub/climate-econometrics/countries/portugal/purchases_paper/notebooks/modeling/panelvar.py:55: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_by_cat = series.groupby(cat).transform("mean")
/Users/dhruvnovaims/Library/Mobile Documents/com~apple~CloudDocs/Documents/GitHub/climate-econometrics/countries/portugal/purchases_paper/notebooks/modeling/panelvar.py:55: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_by_cat = series.groupby(cat).transform("mean")
/Users/dhruvnovaims/Library/Mobile Documents/com~apple~CloudDocs/Documents

KeyboardInterrupt: 